In [528]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil import relativedelta

In [285]:
url = "https://www.sgcarmart.com/used_cars/listing.php?RPG=ALL&MOD=Porsche%20911&VEH=0&AVL=2"
html_data = requests.get(url).text

In [529]:
soup = BeautifulSoup(html_data, 'html5lib')

In [530]:
content = soup.find("div", attrs = {"id": "contentblank"})

In [531]:
listings = content.find_all("table", attrs = {"style": "table-layout: fixed;"})

In [532]:
df = pd.DataFrame(columns = ["PostingDate", "Listing", "Price(SGD)", "Depreciation/yr", "RegDate", 
                             "EngineCapacity(cc)", "Mileage(km)", "VehicleType", "Status"])

for row in range(0, len(listings)):
    try:
        td = listings[row].tbody.find_all(class_ = "font_gray_light font_10")
        div = listings[row].tbody.find_all("div")
        if (div != []): 
            PostingDate = td[0].text.strip('Posted: ')
            listing = div[2].text
            price = div[3].text.strip("\n").strip("\t").strip(" ").strip("$")
            depre = div[4].text.strip("\n").strip(" ").strip("/yr").strip("$").strip(" ")
            reg_date = div[5].text.strip("\n").strip("\t").strip(" ").strip("\n").strip(" ")
            eng_cap = div[6].text.strip("\n").strip("\t").strip(" ").strip("cc").strip(" ")
            mileage = div[7].text.strip("\n").strip("\t").strip(" ").strip("km").strip(" ")
            veh_type = div[8].text.strip("\n").strip("\t").strip(" ")
            status = div[9].text.strip("\n").strip("\t").strip(" ")
            #append the data of each row to the table
            df = df.append({"PostingDate": PostingDate, "Listing": listing, "Price(SGD)": price,
                            "Depreciation/yr": depre, "RegDate": reg_date, 
                            "EngineCapacity(cc)": eng_cap, "Mileage(km)": mileage, 
                            "VehicleType": veh_type, "Status": status},
                            ignore_index = True)
    except:
        continue


In [533]:
df

,PostingDate,Listing,Price(SGD),Depreciation/yr,RegDate,EngineCapacity(cc),Mileage(km),VehicleType,Status
0,30-Dec-2021,Porsche 911 Carrera Coupe 3.0A PDK,"393,800","59,660",25-Jan-2017,"2,981","24,000",Sports,Available
1,30-Dec-2021,Porsche 911 Carrera GTS Coupe 3.8A PDK (COE ti...,"312,000","34,310",26-Jul-2011,"3,800","85,000",Sports,Available
2,30-Dec-2021,Porsche 911 GT3 4.0M,"730,000","81,990",12-Apr-2019,"3,996","14,000",Sports,Available
3,30-Dec-2021,Porsche 911 Carrera Coupe 3.0A PDK,"528,800","53,250",27-Feb-2020,"2,981","18,800",Sports,Available
4,29-Dec-2021,Porsche 911 Carrera Coupe 3.6A PDK (COE till 0...,"248,888","30,320",15-Mar-2010,"3,614","108,000",Sports,Available
...,...,...,...,...,...,...,...,...,...
124,29-Oct-2021,Porsche 911 Carrera Coupe 3.0A PDK,"603,988","50,920",23-Aug-2021,"2,981",652,Sports,Available
125,29-Oct-2021,Porsche 911 Carrera Coupe 3.0A PDK,"587,988",N.A,N.A.,"2,981",50,Sports,Available
126,28-Oct-2021,Porsche 911 Carrera Convertible 3.0A PDK,"639,988",N.A,N.A.,"2,981",50,Sports,Available
127,28-Oct-2021,Porsche 911 Carrera S Coupe 3.0A PDK,"642,988","57,140",06-Feb-2021,"2,981",69,Sports,Available


In [534]:
#removing "," from numbers and converting to integer
target_cols = ['Price(SGD)', 'Depreciation/yr', 'EngineCapacity(cc)', 'Mileage(km)']

for col in target_cols:
    for row in range(0, len(df)):
        df.loc[row, col] = re.sub(",", "", df.loc[row, col]) #remove "," from numbers
    #convert column to numeric once above loop is completed
    #errors = "coerce" to pick-up values like "N.A" and "-" and cast to NaN (blank)
    #downcast = "float" to use the most compact numeric type
    df[col] = pd.to_numeric(df[col], errors = "coerce", downcast = "float") 
    
#convert PostingDate and RegDate to datetime-format
date_cols = ['PostingDate', 'RegDate']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format = '%d-%b-%Y', errors = "coerce")

In [535]:
#Creating new columns for PostingAge(Months), CarAge(Years) and CarAge(Months)

#getting the current datetime
now = datetime.now()


#create a boolean list "m" and "n" to test for NaN values
df['Temp'] = now
m = df[['Temp', 'PostingDate']].notnull().all(axis = 1)
#calculating the PostingAge
df['TotalPostingAge(Days)'] = df['Temp'] - df['PostingDate']
df.loc[m, 'PostingAge(Months)'] = df[m].apply(lambda x: relativedelta.relativedelta(x['Temp'], x['PostingDate']).months, axis = 1)

#calculating the CarAge
n = df[['Temp', 'RegDate']].notnull().all(axis = 1)
df['TotalCarAge(Days)'] = df['Temp'] - df['RegDate']
df.loc[n, 'CarAge(Years)'] = df[n].apply(lambda x: relativedelta.relativedelta(x['Temp'], x['RegDate']).years, axis = 1)
df.loc[n, 'CarAge(Months)'] = df[n].apply(lambda x: relativedelta.relativedelta(x['Temp'], x['RegDate']).months, axis = 1)

#dropping the 'Temp' column from the dataframe
df.drop('Temp', axis = 1, inplace = True)

In [536]:
df

,PostingDate,Listing,Price(SGD),Depreciation/yr,RegDate,EngineCapacity(cc),Mileage(km),VehicleType,Status,TotalPostingAge(Days),PostingAge(Months),TotalCarAge(Days),CarAge(Years),CarAge(Months)
0,2021-12-30,Porsche 911 Carrera Coupe 3.0A PDK,393800.0,59660.0,2017-01-25,2981.0,24000.0,Sports,Available,0 days 13:21:17.877697,0,1800 days 13:21:17.877697,4.0,11.0
1,2021-12-30,Porsche 911 Carrera GTS Coupe 3.8A PDK (COE ti...,312000.0,34310.0,2011-07-26,3800.0,85000.0,Sports,Available,0 days 13:21:17.877697,0,3810 days 13:21:17.877697,10.0,5.0
2,2021-12-30,Porsche 911 GT3 4.0M,730000.0,81990.0,2019-04-12,3996.0,14000.0,Sports,Available,0 days 13:21:17.877697,0,993 days 13:21:17.877697,2.0,8.0
3,2021-12-30,Porsche 911 Carrera Coupe 3.0A PDK,528800.0,53250.0,2020-02-27,2981.0,18800.0,Sports,Available,0 days 13:21:17.877697,0,672 days 13:21:17.877697,1.0,10.0
4,2021-12-29,Porsche 911 Carrera Coupe 3.6A PDK (COE till 0...,248888.0,30320.0,2010-03-15,3614.0,108000.0,Sports,Available,1 days 13:21:17.877697,0,4308 days 13:21:17.877697,11.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2021-10-29,Porsche 911 Carrera Coupe 3.0A PDK,603988.0,50920.0,2021-08-23,2981.0,652.0,Sports,Available,62 days 13:21:17.877697,2,129 days 13:21:17.877697,0.0,4.0
125,2021-10-29,Porsche 911 Carrera Coupe 3.0A PDK,587988.0,NaN,NaT,2981.0,50.0,Sports,Available,62 days 13:21:17.877697,2,NaT,NaN,NaN
126,2021-10-28,Porsche 911 Carrera Convertible 3.0A PDK,639988.0,NaN,NaT,2981.0,50.0,Sports,Available,63 days 13:21:17.877697,2,NaT,NaN,NaN
127,2021-10-28,Porsche 911 Carrera S Coupe 3.0A PDK,642988.0,57140.0,2021-02-06,2981.0,69.0,Sports,Available,63 days 13:21:17.877697,2,327 days 13:21:17.877697,0.0,10.0


In [537]:
df.describe()

,Price(SGD),Depreciation/yr,EngineCapacity(cc),Mileage(km),TotalPostingAge(Days),PostingAge(Months),TotalCarAge(Days),CarAge(Years),CarAge(Months)
count,1.290000e+02,117.000000,129.000000,118.000000,129,129.000000,122,122.000000,122.000000
mean,4.582690e+05,64968.207031,3479.875977,52169.472656,25 days 11:29:40.203278395,0.403101,3463 days 06:04:34.599008512,8.893443,6.500000
std,2.358648e+05,49096.550781,365.104065,49932.910156,19 days 00:19:53.690850337,0.579862,2977 days 07:06:13.166444608,8.163421,3.296003
min,1.268000e+05,21220.000000,2650.000000,11.000000,0 days 13:21:17.877697,0.000000,6 days 13:21:17.877697,0.000000,0.000000
25%,2.888880e+05,35360.000000,2981.000000,4897.000000,9 days 13:21:17.877697,0.000000,692 days 19:21:17.877697,1.000000,4.000000
50%,3.938000e+05,51620.000000,3600.000000,39500.000000,21 days 13:21:17.877697,0.000000,3506 days 13:21:17.877697024,9.000000,7.000000
75%,6.039880e+05,70060.000000,3800.000000,86600.000000,38 days 13:21:17.877697,1.000000,4584 days 19:21:17.877697024,12.000000,9.000000
max,1.300000e+06,320480.000000,3996.000000,190000.000000,64 days 13:21:17.877697,2.000000,15452 days 13:21:17.877697,42.000000,11.000000
